In [ ]:
# ! pip install python-dotenv
# ! pip install langchain --upgrade --no-cache-dir
! pip install wandb --upgrade --no-cache-dir

In [ ]:
import wandb
wandb.__version__

In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload

import sys
import os
import json

os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "langchain-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" ) 
    
print( sys.path )

import util as du
path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import lib.util_stopwatch as sw
import lib.util_langchain as ulc
import lib.util_pandas    as up

import genie_client as gc

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

import pandas as pd


from IPython.display import display, Markdown


[/var/genie-in-the-box/src/lib] already in sys.path
['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/var/genie-in-the-box/src/lib', '/var/genie-in-the-box/src']
Path [/var/genie-in-the-box/src] already in sys.path


In [24]:
raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
print( raw_df.info() )

raw_df = up.cast_to_datetime( raw_df )

raw_df[ 'recurrence_interval' ] = raw_df[ 'recurrence_interval' ].fillna( '' )
raw_df.loc[ raw_df.recurrence_interval.str.endswith( "year" ), "recurrence_interval" ] = "1 year"

print( raw_df.dtypes )
raw_df.to_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv", columns=raw_df.columns, index=False )


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   start_date                  200 non-null    object
 1   end_date                    200 non-null    object
 2   start_time                  200 non-null    object
 3   end_time                    200 non-null    object
 4   event_type                  200 non-null    object
 5   recurrent                   200 non-null    bool  
 6   recurrence_interval         47 non-null     object
 7   priority_level              200 non-null    object
 8   name                        200 non-null    object
 9   relationship                200 non-null    object
 10  description_who_what_where  200 non-null    object
dtypes: bool(1), object(10)
memory usage: 15.9+ KB
None
start_date                    datetime64[ns]
end_date                      datetime64[ns]
start_time           

In [22]:
print(raw_df.dtypes)

start_date                    datetime64[ns]
end_date                      datetime64[ns]
start_time                            object
end_time                              object
event_type                            object
recurrent                               bool
recurrence_interval                   object
priority_level                        object
name                                  object
relationship                          object
description_who_what_where            object
dtype: object


In [15]:
raw_df = up.cast_to_datetime( raw_df )
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   start_date                  200 non-null    datetime64[ns]
 1   end_date                    200 non-null    datetime64[ns]
 2   start_time                  200 non-null    object        
 3   end_time                    200 non-null    object        
 4   event_type                  200 non-null    object        
 5   recurrent                   200 non-null    bool          
 6   recurrence_interval         47 non-null     object        
 7   priority_level              200 non-null    object        
 8   name                        200 non-null    object        
 9   relationship                200 non-null    object        
 10  description_who_what_where  200 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(8)
memory usage: 15.

In [4]:
langchain.debug = True
langchain.verbose = True

llm_4   = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
llm_3_5 = ChatOpenAI( model_name="gpt-3.5-turbo-0613", temperature=0.0 )
memory  = ConversationBufferMemory( memory_key="chat_history", return_messages=True, verbose=True )

df_agent = create_pandas_dataframe_agent( llm_4, raw_df, verbose=True )
df_agent.memory = memory

In [36]:
"foo {bar} baz {bar}".format( bar="bar" )

'foo bar baz bar'

In [66]:
def get_question_and_coda( event_query ):
    
    question_plus_coda_template = f"""
    {event_query}  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: The fields `start_date` and `end_date` are stored as strings. Make sure that you use the same data types when comparing or filtering dates.
    Hint: If your code returns any records, they should include all columns in the dataframe and be returned using a print statement
    
    You must verify that your code runs to completion, once it does, return your response as a JSON string containing only these three fields: 
    1) answer: A simple, terse answer to the question '{event_query}'
    2) code: The Python code you generated in python list form, one line of code per item in the list, and 
    3) explanation: A brief explanation as to how the code works"""
    
    question = question_plus_coda_template.format( event_query=event_query )
    return question

question = get_question_and_coda( "What events do I have today?" )
for line in question.split( "\n" ): 
    print( line )


    What events do I have today?  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: The fields `start_date` and `end_date` are stored as strings. Make sure that you use the same data types when comparing or filtering dates.
    Hint: If your code returns any records, they should include all columns in the dataframe and be returned using a print statement
    
    You must verify that your code runs to completion, once it does, return your response as a JSON string containing only these three fields: 
    1) answer: A simple, terse answer to the question 'What events do I have today?'
    2) code: The Python code you generated in python list form, one line of code per item in the list, and 
    3) explanation: A brief explanation as to how the code works


In [53]:

question = get_question_and_coda( "What events do I have today?" )

timer = sw.Stopwatch( "Running pandas prompt..." )
agent_response = df_agent.run( question )
timer.print( "Done!" )

Running pandas prompt...
[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n

In [61]:
for line in agent_response.split( "\n" ):
    print( line )

```json
{
    "answer": "[{\"start_date\":1693440000000,\"end_date\":1694044800000,\"start_time\":\"04:11\",\"end_time\":\"16:53\",\"event_type\":\"Workshop\",\"recurrent\":false,\"recurrence_interval\":null,\"priority_level\":\"none\",\"name\":\"John\",\"relationship\":\"coworker\",\"description_who_what_where\":\"Exciting workshop with John on AGI\"}]",
    "code": [
        "import pandas as pd",
        "from datetime import datetime",
        "df['start_date'] = pd.to_datetime(df['start_date'])",
        "df['end_date'] = pd.to_datetime(df['end_date'])",
        "today = pd.to_datetime(datetime.now().date())",
        "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
        "answer = events_today.to_json(orient='records')"
    ],
    "explanation": "The code first converts the start_date and end_date columns in the dataframe to datetime objects. It then gets today's date as a datetime object. It filters the dataframe to only include rows where today's 

In [62]:
stripped_response = "{" + agent_response.split( "{" )[ 1 ].split( "}" )[ -1 ] + "}"
stripped_response

'{\n    "answer": "[}'

In [63]:
stripped_response = agent_response.strip( "```" ).strip( "json" ).strip( "\n" )
stripped_response

'{\n    "answer": "[{\\"start_date\\":1693440000000,\\"end_date\\":1694044800000,\\"start_time\\":\\"04:11\\",\\"end_time\\":\\"16:53\\",\\"event_type\\":\\"Workshop\\",\\"recurrent\\":false,\\"recurrence_interval\\":null,\\"priority_level\\":\\"none\\",\\"name\\":\\"John\\",\\"relationship\\":\\"coworker\\",\\"description_who_what_where\\":\\"Exciting workshop with John on AGI\\"}]",\n    "code": [\n        "import pandas as pd",\n        "from datetime import datetime",\n        "df[\'start_date\'] = pd.to_datetime(df[\'start_date\'])",\n        "df[\'end_date\'] = pd.to_datetime(df[\'end_date\'])",\n        "today = pd.to_datetime(datetime.now().date())",\n        "events_today = df[(df[\'start_date\'] <= today) & (df[\'end_date\'] >= today)]",\n        "answer = events_today.to_json(orient=\'records\')"\n    ],\n    "explanation": "The code first converts the start_date and end_date columns in the dataframe to datetime objects. It then gets today\'s date as a datetime object. It fi

In [64]:
def get_clean_dictionary( response ):

    # Kludgy way to strip off anything before and after the opening and closing curly brackets
    # stripped_response = "{" + response.split( "{" )[ 1 ].split( "}" )[ 0 ] + "}"
    stripped_response = agent_response.strip( "```" ).strip( "json" ).strip( "\n" )
    
    return json.loads( stripped_response )
    
agent_response_dict = get_clean_dictionary( agent_response[ : ] )

In [65]:
agent_response_dict

{'answer': '[{"start_date":1693440000000,"end_date":1694044800000,"start_time":"04:11","end_time":"16:53","event_type":"Workshop","recurrent":false,"recurrence_interval":null,"priority_level":"none","name":"John","relationship":"coworker","description_who_what_where":"Exciting workshop with John on AGI"}]',
 'code': ['import pandas as pd',
  'from datetime import datetime',
  "df['start_date'] = pd.to_datetime(df['start_date'])",
  "df['end_date'] = pd.to_datetime(df['end_date'])",
  'today = pd.to_datetime(datetime.now().date())',
  "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
  "answer = events_today.to_json(orient='records')"],
 'explanation': "The code first converts the start_date and end_date columns in the dataframe to datetime objects. It then gets today's date as a datetime object. It filters the dataframe to only include rows where today's date falls within the start and end dates of the event. Finally, it converts this filtered dataframe to a

In [59]:
response = ulc.assemble_and_run_solution( agent_response_dict[ "code" ], path=du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

for line in response[ "response" ].split( "\n" ):
    print( line )

ERROR:
Traceback (most recent call last):
  File "/var/genie-in-the-box/io/code.py", line 9, in <module>
    print( answer = events_today.to_json(orient='records') )
TypeError: 'answer' is an invalid keyword argument for print()


In [ ]:
genie_client = gc.GenieClient()
genie_client.ask_chat_gpt_text( response[ "response" ], preamble="This is output from a query on a pandas dataframe. Rephrase this output in conversational English in such a way that answers the question: what events do I have today?" )